# Lines Analysis

## Import

In [1]:
import os
import uuid

import numpy as np
import pandas as pd

import janitor

## Constants

In [2]:
scores = ['GOAL', 'SCORED_ON', 'CALLAHAN', 'CALLAHAN_THROWN']
we_score = ['GOAL', 'CALLAHAN']
they_score = ['SCORED_ON', 'CALLAHAN_THROWN']

turnovers = ['THROWAWAY', 'DROP', 'STALL', 'CALLAHAN_THROWN']
takeaways = ['BLOCK', 'CALLAHAN', 'STALL_CAUSED', 'THROWAWAY_CAUSED']

pulls = ['PULL_OUT_OF_BOUNDS', 'PULL_INBOUNDS']

pens = ['D_PENALTY_ON_THEM', 'D_PENALTY_ON_US', 'O_PENALTY_ON_THEM', 'O_PENALTY_ON_US']

quarter_ends = ['START_OF_GAME', 'END_OF_Q1', 'HALFTIME', 'END_OF_Q3', 'GAME_OVER']

line_sets = ['SET_D_LINE', 'SET_O_LINE', 'SET_D_LINE_NO_PULL', 'SET_O_LINE_NO_PULL']
o_lines = ['SET_O_LINE', 'SET_O_LINE_NO_PULL']
d_lines = ['SET_D_LINE', 'SET_D_LINE_NO_PULL']

timeouts = ['THEIR_MIDPOINT_TIMEOUT', 'OUR_MIDPOINT_TIMEOUT']

other = ['REF_TIMEOUT_DISCUSSION???', 'INJURY_ON_O', 'INJURY_ON_D']


## Analysis

### File IO

In [3]:
df = pd.read_csv("Flyers_Games.csv").rename({"Unnamed: 0" : "event_index"}, axis=1)
df_home = pd.read_csv("Flyers_Games_home.csv").rename({"Unnamed: 0" : "event_index"}, axis=1)
df_away = pd.read_csv("Flyers_Games_away.csv").rename({"Unnamed: 0" : "event_index"}, axis=1)

In [4]:
df = pd.read_csv("All_Games_220806_2200.csv").rename({"Unnamed: 0" : "event_index"}, axis=1)

In [5]:
df.head(35)

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,player,x,y,o_point,d_point,point_id,our_score,their_score,line
0,0,2836,2022-07-02,244,240,0,244,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,0,NaN
1,1,2836,2022-07-02,244,240,1,244,NaN,NaN,NaN,NaN,NaN,NaN,False,False,0,0,0,NaN
2,2,2836,2022-07-02,244,240,2,244,1.0,NaN,START_OF_GAME,NaN,NaN,NaN,False,False,0,0,0,NaN
3,3,2836,2022-07-02,244,240,3,244,1.0,NaN,SET_D_LINE,"Austin Lillis, Eric Witmer, Eric Nardelli, Pau...",NaN,NaN,False,True,1,0,0,"['Austin Lillis', 'Eric Witmer', 'Eric Nardell..."
4,4,2836,2022-07-02,244,240,4,244,1.0,NaN,PULL_INBOUNDS,Paul Owens,-23.98,58.39,False,True,1,0,0,NaN
5,5,2836,2022-07-02,244,240,5,244,1.0,Q1 11:38,SCORED_ON,NaN,NaN,NaN,False,True,1,0,1,NaN
6,6,2836,2022-07-02,244,240,6,244,1.0,NaN,SET_O_LINE,"Dustin Damiano, Greg Martin, Sean Mott, Alex T...",NaN,NaN,True,False,2,0,1,"['Dustin Damiano', 'Greg Martin', 'Sean Mott',..."
7,7,2836,2022-07-02,244,240,7,244,1.0,NaN,NaN,NaN,NaN,NaN,True,False,2,0,1,NaN
8,8,2836,2022-07-02,244,240,8,244,1.0,NaN,NaN,NaN,NaN,NaN,True,False,2,0,1,NaN
9,9,2836,2022-07-02,244,240,9,244,1.0,NaN,POSSESSION,Alex Thorne,5.24,52.71,True,False,2,0,1,NaN


### Cleaning

In [6]:
df['is_home'] = df['home_team'] == df['team_id']
df['is_away'] = df['away_team'] == df['team_id']

df["game_point_id"] = df["game_id"]*1000 + df["point_id"]

df['player'] = np.where(df['event_type'].isin(line_sets), df['player'], "")

df['home_score']  = np.where((df['is_home']) & (df['event_type'].isin(we_score)), 1, 0)
df['away_score']  = np.where((df['is_away']) & (df['event_type'].isin(we_score)), 1, 0)

### LINES

In [7]:
all_lines = df.loc[df['event_type'].isin(line_sets)].copy()

# How many lines are there per point
all_lines['line_number_in_point'] = all_lines.groupby('game_point_id')['event_counter'].cumcount()
print("Max Number of Lines in a point:", all_lines['line_number_in_point'].unique().max())

# Make Line ID within point within game
# Multiply by 10 is safe
all_lines['game_point_line_id'] = all_lines['game_point_id']*10 + all_lines['line_number_in_point']



Max Number of Lines in a point: 10


In [8]:
all_lines.columns

Index(['event_index', 'game_id', 'date', 'home_team', 'away_team',
       'event_counter', 'team_id', 'current_quarter', 'time', 'event_type',
       'player', 'x', 'y', 'o_point', 'd_point', 'point_id', 'our_score',
       'their_score', 'line', 'is_home', 'is_away', 'game_point_id',
       'home_score', 'away_score', 'line_number_in_point',
       'game_point_line_id'],
      dtype='object')

In [9]:
# Players in each line
all_lines.drop(all_lines.loc[all_lines['player'].isna()].index, inplace=True)
all_lines['line_players_original'] = [[x.strip() for x in y[:-1]] for y in all_lines['player'].str.split(",")]
all_lines['line_players'] = all_lines['line_players_original'].apply(lambda x: sorted(x))

all_lines['line_players'].value_counts()

[Cameron Brock, Carter Rae, Jeremy Keusch, Keegan North, Levi Jacobs, Lucas Coniaris, Rick Gross]                       59
[Benjamin Sokolowsky, Chris Lung, Justin Norden, Matthew Kissmann, Max Williams, Morgan Sommer, Robert Yeagle]          56
[Alex Davis, Eric Taylor, Henry Fisher, Jacob Fairfax, Matt Gouchoe-Hanas, Sol Yanuck, Terrence Mitchell]               50
[Jakob Brissette, Kevin Quinlan, Malik Auger-Semmar, Miguel Goderre, Nabil Chaouch, Quentin Bonnaud, Thomas Duplaix]    30
[Daniel Brunker, Danny Landesman, Jonathan Nethercutt, Matt Jackson, Nathan Buchholz, Quinn Finer, Thomas Brewster]     29
                                                                                                                        ..
[Cody Spicer, Conor Tabor, Danny Landesman, Jonathan Nethercutt, Matt Jackson, Quinn Finer, Thomas Brewster]             1
[Asher Lantz, Charlie Furse, Jace Bruner, Jason Vallee, John Jones, Matt Jaffe, Nick Pappas]                             1
[Asher Lantz, Ja

In [10]:
# ID for each unique line
deduped_lines = all_lines['line_players'].drop_duplicates()
deduped_idx = deduped_lines.index

df_line_ids = pd.concat([all_lines.loc[deduped_idx, 'team_id'], deduped_lines], axis=1).reset_index(drop=True)
df_line_ids['line_id'] = df['team_id']*10000 + df_line_ids.groupby('team_id')['line_players'].transform('cumcount')
df_line_ids['line_id'] = df_line_ids['line_id'].astype(int)
df_line_ids

,team_id,line_players,line_id
0,244,"[Austin Lillis, Eric Nardelli, Eric Witmer, Ga...",2440000
1,244,"[Alex Thorne, Anthony Prato, Dmitry Suvorov, D...",2440001
2,244,"[Adam Grossberg, Eric Witmer, Marc Sands, Matt...",2440002
3,244,"[Alex Thorne, Anthony Prato, Brandon Pastor, D...",2440003
4,244,"[Adam Grossberg, Eric Witmer, Marc Sands, Matt...",2440004
...,...,...,...
3178,240,"[AJ Merriman, Alexandre Fall, David Bloodgood,...",2410192
3179,240,"[AJ Merriman, Alexandre Fall, David Bloodgood,...",2410193
3180,240,"[AJ Merriman, Alexandre Fall, David Bloodgood,...",2410194
3181,240,"[AJ Merriman, Alexandre Fall, David Bloodgood,...",2410195


In [11]:
# Need to get line ids
df_line_sets = all_lines.loc[:, ['team_id', 'game_id', 'line_players']]

# result = df.a.sort_values().apply(lambda x: sorted(x))

df_line_sets['sorted_line_players'] = df_line_sets['line_players'].apply(lambda x: sorted(x))
split_series = df_line_sets.sorted_line_players.apply(lambda x:pd.Series(str(x).replace("[", "").replace("]", "").split(",")))
print(split_series.shape)
split_series.drop_duplicates()

# ser = pd.get_dummies(df_line_sets['line_players'])

(6950, 7)


,0,1,2,3,4,5,6
3,'Austin Lillis','Eric Nardelli','Eric Witmer','Gavin Abrahamsson','Mike Campanella','Paul Owens','Scott Heyman'
6,'Alex Thorne','Anthony Prato','Dmitry Suvorov','Dustin Damiano','Greg Martin','James Pollard','Sean Mott'
14,'Adam Grossberg','Eric Witmer','Marc Sands','Matt Esser','Matt Hanna','Nate Little','Paul Owens'
17,'Alex Thorne','Anthony Prato','Brandon Pastor','Dustin Damiano','Greg Martin','James Pollard','Sean Mott'
52,'Adam Grossberg','Eric Witmer','Marc Sands','Matt Esser','Matt Hanna','Nate Little','Scott Heyman'
...,...,...,...,...,...,...,...
60456,'AJ Merriman','Alexandre Fall','David Bloodgood','Isaac Lee','Kyle Vezina','Rhys Bergeron','Tyler Monroe'
60465,'AJ Merriman','Alexandre Fall','David Bloodgood','Isaac Lee','Jasper Tom','Marcus Thaw','Moussa Dia'
60481,'AJ Merriman','Alexandre Fall','David Bloodgood','Frederick Farah','Jasper Tom','Moussa Dia','Rhys Bergeron'
60512,'AJ Merriman','Alexandre Fall','David Bloodgood','Isaac Lee','Kevin Healey','Marcus Thaw','Moussa Dia'


In [12]:
df_line_sets = df_line_sets.merge(
    split_series, left_index=True, 
    right_index=True, copy=True
)

df_line_sets['line_id'] = [uuid.uuid4() for _ in range(len(df_line_sets.index))]

In [13]:
# Last Line on for each point
last_line_by_point_idx = all_lines.groupby(['game_point_id', 'team_id'], sort=False)['game_point_line_id'].transform('max') == all_lines['game_point_line_id']
last_lines = all_lines.loc[last_line_by_point_idx].sort_values('game_point_id')
# last_lines.sort_values(['game_point_id', 'game_point_line_id']).head(5)

all_lines = all_lines.merge(last_lines.loc[:, 'game_point_line_id'], how='left', on='game_point_line_id', indicator=True)
all_lines['is_last_line'] = all_lines['_merge'] == 'both'
del all_lines['_merge']

In [14]:
home_lines = all_lines.loc[all_lines['is_home'] == True].copy()
away_lines = all_lines.loc[all_lines['is_home'] == False].copy()

In [15]:
all_lines.columns

Index(['event_index', 'game_id', 'date', 'home_team', 'away_team',
       'event_counter', 'team_id', 'current_quarter', 'time', 'event_type',
       'player', 'x', 'y', 'o_point', 'd_point', 'point_id', 'our_score',
       'their_score', 'line', 'is_home', 'is_away', 'game_point_id',
       'home_score', 'away_score', 'line_number_in_point',
       'game_point_line_id', 'line_players_original', 'line_players',
       'is_last_line'],
      dtype='object')

In [16]:
keep_cols_a = [
    'date', 'game_id', 'game_point_id', 'game_point_line_id', 
    'home_team', 'away_team', 'team_id', 'current_quarter',
    'event_type', 'o_point', 'd_point', 'point_id',
    'our_score', 'their_score', 'is_home', 'is_away', 'line_players',
    'is_last_line', 'line_number_in_point'
]

keep_cols_b = ['game_point_id', 'game_point_line_id', 'team_id', 'line_players', 'event_type', 'is_last_line']

rename_cols_a = {
    'team_id': 'team_id_a',
    'is_home': 'is_home_a',
    'is_away': 'is_away_a',
    'game_point_line_id': 'line_id_a',
    'line_players': 'line_players_a',
    'is_last_line': 'is_last_line_a',
    'our_score': 'our_score_point_start',
    'their_score': 'their_score_point_start',
    'event_type': 'line_set_type_a',
    'o_point': 'o_point_a'
}

rename_cols_b = {
    'team_id': 'team_id_b',
    'game_point_line_id': 'line_id_b',
    'line_players': 'line_players_b',
    'is_last_line': 'is_last_line_b',
    'event_type': 'line_set_type_b',
}

final_lines = home_lines.loc[:, keep_cols_a].rename(rename_cols_a, axis=1).merge(
    away_lines.loc[:, keep_cols_b].rename(rename_cols_b, axis=1),
    how='outer',
    on='game_point_id',
    indicator=True,
    copy=True
)
del final_lines['_merge']
print(final_lines.shape)
final_lines.head()

(4443, 24)


,date,game_id,game_point_id,line_id_a,home_team,away_team,team_id_a,current_quarter,line_set_type_a,o_point_a,...,is_home_a,is_away_a,line_players_a,is_last_line_a,line_number_in_point,line_id_b,team_id_b,line_players_b,line_set_type_b,is_last_line_b
0,2022-07-02,2836.0,2836001,28360010.0,244.0,240.0,244.0,1.0,SET_D_LINE,False,...,True,False,"[Austin Lillis, Eric Nardelli, Eric Witmer, Ga...",True,0.0,28360011.0,240.0,"[Christian Boxley, Jacques Nissen, Jonny Malks...",SET_O_LINE,True
1,2022-07-02,2836.0,2836002,28360020.0,244.0,240.0,244.0,1.0,SET_O_LINE,True,...,True,False,"[Alex Thorne, Anthony Prato, Dmitry Suvorov, D...",True,0.0,28360021.0,240.0,"[AJ Merriman, Aaron Bartlett, Alexandre Fall, ...",SET_D_LINE,True
2,2022-07-02,2836.0,2836003,28360030.0,244.0,240.0,244.0,1.0,SET_D_LINE,False,...,True,False,"[Adam Grossberg, Eric Witmer, Marc Sands, Matt...",True,0.0,28360031.0,240.0,"[Christian Boxley, Jacques Nissen, Jeremy Knop...",SET_O_LINE,True
3,2022-07-02,2836.0,2836004,28360040.0,244.0,240.0,244.0,1.0,SET_O_LINE,True,...,True,False,"[Alex Thorne, Anthony Prato, Brandon Pastor, D...",True,0.0,28360041.0,240.0,"[Aaron Bartlett, Brandon Lamberty, David Blood...",SET_D_LINE,True
4,2022-07-02,2836.0,2836005,28360050.0,244.0,240.0,244.0,1.0,SET_D_LINE,False,...,True,False,"[Austin Lillis, Eric Nardelli, Eric Witmer, Ga...",True,0.0,28360051.0,240.0,"[Christian Boxley, Jacques Nissen, Jeremy Knop...",SET_O_LINE,True


In [17]:
df_quarter_ends = df.loc[(df['event_type'].isin(quarter_ends))].copy()
df_quarter_ends.shape
df_quarter_ends.head()

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,...,d_point,point_id,our_score,their_score,line,is_home,is_away,game_point_id,home_score,away_score
2,2,2836,2022-07-02,244,240,2,244,1.0,NaN,START_OF_GAME,...,False,0,0,0,NaN,True,False,2836000,0,0
109,109,2836,2022-07-02,244,240,109,244,2.0,NaN,END_OF_Q1,...,True,12,6,5,NaN,True,False,2836012,0,0
211,211,2836,2022-07-02,244,240,211,244,3.0,NaN,HALFTIME,...,True,22,10,11,NaN,True,False,2836022,0,0
326,326,2836,2022-07-02,244,240,326,244,4.0,NaN,END_OF_Q3,...,False,31,14,15,NaN,True,False,2836031,0,0
439,439,2836,2022-07-02,244,240,439,244,4.0,NaN,GAME_OVER,...,False,46,21,22,NaN,True,False,2836046,0,0


### POINTS

In [18]:
df_scores = df.loc[(df['event_type'].isin(we_score))].copy()

df_scores['scoring_team'] = df_scores['team_id']
# df_scores = df.loc[(df['event_type'] == 'GOAL') | (df['event_type'] == 'CALLAHAN')].copy()
df_scores.sort_values('game_point_id').head(10)

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,...,point_id,our_score,their_score,line,is_home,is_away,game_point_id,home_score,away_score,scoring_team
32411,7,2807,2022-07-02,243,247,7,247,1.0,Q1 11:25,GOAL,...,1,1,0,NaN,False,True,2807001,0,1,247
32431,27,2807,2022-07-02,243,247,27,247,1.0,Q1 09:29,GOAL,...,2,2,0,NaN,False,True,2807002,0,1,247
1867,29,2807,2022-07-02,243,247,29,243,1.0,Q1 08:45,GOAL,...,3,1,2,NaN,True,False,2807003,1,0,243
32450,46,2807,2022-07-02,243,247,46,247,1.0,Q1 07:38,GOAL,...,4,3,1,NaN,False,True,2807004,0,1,247
32457,53,2807,2022-07-02,243,247,53,247,1.0,Q1 05:56,GOAL,...,5,4,1,NaN,False,True,2807005,0,1,247
32465,61,2807,2022-07-02,243,247,61,247,1.0,Q1 04:43,GOAL,...,6,5,1,NaN,False,True,2807006,0,1,247
1910,72,2807,2022-07-02,243,247,72,243,1.0,Q1 03:51,GOAL,...,7,2,5,NaN,True,False,2807007,1,0,243
1928,90,2807,2022-07-02,243,247,90,243,1.0,Q1 01:42,GOAL,...,8,3,5,NaN,True,False,2807008,1,0,243
32488,84,2807,2022-07-02,243,247,84,247,1.0,Q1 01:32,GOAL,...,9,6,3,NaN,False,True,2807009,0,1,247
1942,104,2807,2022-07-02,243,247,104,243,1.0,Q1 00:51,GOAL,...,10,4,6,NaN,True,False,2807010,1,0,243


In [19]:
df_scores.columns

Index(['event_index', 'game_id', 'date', 'home_team', 'away_team',
       'event_counter', 'team_id', 'current_quarter', 'time', 'event_type',
       'player', 'x', 'y', 'o_point', 'd_point', 'point_id', 'our_score',
       'their_score', 'line', 'is_home', 'is_away', 'game_point_id',
       'home_score', 'away_score', 'scoring_team'],
      dtype='object')

In [20]:
score_keep_cols = [
     'game_point_id', 'time', 'event_type', 'o_point', 'is_home', 'is_away', 
    'our_score', 'their_score', 'scoring_team', 'home_score', 'away_score'
]

score_rename_cols = {
    'event_type': 'score_type',
    'time': 'time_of_score',
    'our_score': 'our_score_point_end',
    'their_score': 'their_score_point_end',
    'home_score': 'home_team_scored', 
    'away_score': 'away_team_scored'
}
final_scores = df_scores.loc[:, score_keep_cols].rename(score_rename_cols, axis=1).sort_values('game_point_id')

### Turns

In [38]:
df_turns = df.loc[(df['event_type'].isin(turnovers))].copy()

df_turns.sort_values('game_point_id').head(10)

,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,...,d_point,point_id,our_score,their_score,line,is_home,is_away,game_point_id,home_score,away_score
1850,12,2807,2022-07-02,243,247,12,243,1.0,NaN,THROWAWAY,...,False,2,0,1,NaN,True,False,2807002,0,0
1884,46,2807,2022-07-02,243,247,46,243,1.0,NaN,THROWAWAY,...,False,5,1,3,NaN,True,False,2807005,0,0
1900,62,2807,2022-07-02,243,247,62,243,1.0,NaN,THROWAWAY,...,False,6,1,4,NaN,True,False,2807006,0,0
32470,66,2807,2022-07-02,243,247,66,247,1.0,NaN,THROWAWAY,...,True,7,5,1,NaN,False,True,2807007,0,0
1905,67,2807,2022-07-02,243,247,67,243,1.0,NaN,DROP,...,False,7,1,5,NaN,True,False,2807007,0,0
32480,76,2807,2022-07-02,243,247,76,247,1.0,NaN,THROWAWAY,...,False,8,5,2,NaN,False,True,2807008,0,0
32496,92,2807,2022-07-02,243,247,92,247,1.0,NaN,THROWAWAY,...,False,11,6,4,NaN,False,True,2807011,0,0
1950,112,2807,2022-07-02,243,247,112,243,1.0,NaN,THROWAWAY,...,True,11,4,6,NaN,True,False,2807011,0,0
1953,115,2807,2022-07-02,243,247,115,243,1.0,NaN,THROWAWAY,...,True,11,4,6,NaN,True,False,2807011,0,0
32501,97,2807,2022-07-02,243,247,97,247,1.0,NaN,THROWAWAY,...,False,11,6,4,NaN,False,True,2807011,0,0


In [39]:
df_turns.columns

Index(['event_index', 'game_id', 'date', 'home_team', 'away_team',
       'event_counter', 'team_id', 'current_quarter', 'time', 'event_type',
       'player', 'x', 'y', 'o_point', 'd_point', 'point_id', 'our_score',
       'their_score', 'line', 'is_home', 'is_away', 'game_point_id',
       'home_score', 'away_score'],
      dtype='object')

In [77]:
pd.DataFrame(df_turns.groupby(['game_point_id', 'team_id'])['event_index'].agg('count')).reset_index()

,game_point_id,team_id,event_index
0,2807002,243,1
1,2807005,243,1
2,2807006,243,1
3,2807007,243,1
4,2807007,247,1
...,...,...,...
2034,2952044,250,2
2035,2952044,252,2
2036,2952045,250,1
2037,2952045,252,1


In [53]:
pd.DataFrame(df_turns.groupby(['game_point_id', 'team_id', 'event_type'])['event_index'].agg('count')).reset_index()

,game_point_id,team_id,event_type,event_index
0,2807002,243,THROWAWAY,1
1,2807005,243,THROWAWAY,1
2,2807006,243,THROWAWAY,1
3,2807007,243,DROP,1
4,2807007,247,THROWAWAY,1
...,...,...,...,...
2164,2952044,250,THROWAWAY,1
2165,2952044,252,THROWAWAY,2
2166,2952045,250,STALL,1
2167,2952045,252,THROWAWAY,1


### FINAL DF

In [21]:
df_final = final_lines.merge(
    final_scores,
    on='game_point_id',
    how='outer',
    indicator=True,
    copy=True
)

df_final.head(10)
df_final._merge.value_counts()

both          4085
left_only      390
right_only       0
Name: _merge, dtype: int64

In [22]:
df_final.columns

Index(['date', 'game_id', 'game_point_id', 'line_id_a', 'home_team',
       'away_team', 'team_id_a', 'current_quarter', 'line_set_type_a',
       'o_point_a', 'd_point', 'point_id', 'our_score_point_start',
       'their_score_point_start', 'is_home_a', 'is_away_a', 'line_players_a',
       'is_last_line_a', 'line_number_in_point', 'line_id_b', 'team_id_b',
       'line_players_b', 'line_set_type_b', 'is_last_line_b', 'time_of_score',
       'score_type', 'o_point', 'is_home', 'is_away', 'our_score_point_end',
       'their_score_point_end', 'scoring_team', 'home_team_scored',
       'away_team_scored', '_merge'],
      dtype='object')

In [23]:
cols = [
    'date', 'game_id', 'home_team', 'away_team', 'game_point_id', 
    'current_quarter', 'time_of_score',
    'team_id_a', 'line_id_a', 'line_players_a', 'line_set_type_a', 'is_last_line_a', 'is_home_a', 'o_point_a',
    'team_id_b', 'line_id_b', 'line_players_b', 'line_set_type_b', 'is_last_line_b',
    'our_score_point_start',
    'their_score_point_start', 'our_score_point_end', 'their_score_point_end',
    'home_team_scored', 'away_team_scored',
    'scoring_team', 'score_type'
]

final_rename = {
    'our_score_point_start': 'team_a_score_point_start',
    'their_score_point_start': 'team_b_score_point_start'
}

leftover = [x for x in df_final.columns if x not in cols]
leftover

['d_point',
 'point_id',
 'is_away_a',
 'line_number_in_point',
 'o_point',
 'is_home',
 'is_away',
 '_merge']

In [24]:
df_final['team_a_scored'] = (df_final['scoring_team'] == df_final['team_id_a']) & df_final['score_type'].notna()
df_final['team_b_scored'] = (df_final['scoring_team'] == df_final['team_id_b']) & df_final['score_type'].notna()

cols.remove('home_team_scored')
cols.remove('away_team_scored')
cols.remove('our_score_point_end')
cols.remove('their_score_point_end')

cols.append('team_a_scored')
cols.append('team_b_scored')
cols

['date',
 'game_id',
 'home_team',
 'away_team',
 'game_point_id',
 'current_quarter',
 'time_of_score',
 'team_id_a',
 'line_id_a',
 'line_players_a',
 'line_set_type_a',
 'is_last_line_a',
 'is_home_a',
 'o_point_a',
 'team_id_b',
 'line_id_b',
 'line_players_b',
 'line_set_type_b',
 'is_last_line_b',
 'our_score_point_start',
 'their_score_point_start',
 'scoring_team',
 'score_type',
 'team_a_scored',
 'team_b_scored']

In [25]:
df_output = df_final.loc[:, cols].rename(final_rename, axis=1)
df_output.columns

Index(['date', 'game_id', 'home_team', 'away_team', 'game_point_id',
       'current_quarter', 'time_of_score', 'team_id_a', 'line_id_a',
       'line_players_a', 'line_set_type_a', 'is_last_line_a', 'is_home_a',
       'o_point_a', 'team_id_b', 'line_id_b', 'line_players_b',
       'line_set_type_b', 'is_last_line_b', 'team_a_score_point_start',
       'team_b_score_point_start', 'scoring_team', 'score_type',
       'team_a_scored', 'team_b_scored'],
      dtype='object')

In [26]:
len(df_output['home_team'].value_counts().sort_index())
df_output.shape

(4475, 25)

In [27]:
df_output.head()

,date,game_id,home_team,away_team,game_point_id,current_quarter,time_of_score,team_id_a,line_id_a,line_players_a,...,line_id_b,line_players_b,line_set_type_b,is_last_line_b,team_a_score_point_start,team_b_score_point_start,scoring_team,score_type,team_a_scored,team_b_scored
0,2022-07-02,2836.0,244.0,240.0,2836001,1.0,Q1 11:38,244.0,28360010.0,"[Austin Lillis, Eric Nardelli, Eric Witmer, Ga...",...,28360011.0,"[Christian Boxley, Jacques Nissen, Jonny Malks...",SET_O_LINE,True,0.0,0.0,240.0,GOAL,False,True
1,2022-07-02,2836.0,244.0,240.0,2836002,1.0,Q1 10:49,244.0,28360020.0,"[Alex Thorne, Anthony Prato, Dmitry Suvorov, D...",...,28360021.0,"[AJ Merriman, Aaron Bartlett, Alexandre Fall, ...",SET_D_LINE,True,0.0,1.0,244.0,GOAL,True,False
2,2022-07-02,2836.0,244.0,240.0,2836003,1.0,Q1 10:30,244.0,28360030.0,"[Adam Grossberg, Eric Witmer, Marc Sands, Matt...",...,28360031.0,"[Christian Boxley, Jacques Nissen, Jeremy Knop...",SET_O_LINE,True,1.0,1.0,240.0,GOAL,False,True
3,2022-07-02,2836.0,244.0,240.0,2836004,1.0,Q1 09:42,244.0,28360040.0,"[Alex Thorne, Anthony Prato, Brandon Pastor, D...",...,28360041.0,"[Aaron Bartlett, Brandon Lamberty, David Blood...",SET_D_LINE,True,1.0,2.0,244.0,GOAL,True,False
4,2022-07-02,2836.0,244.0,240.0,2836005,1.0,Q1 06:56,244.0,28360050.0,"[Austin Lillis, Eric Nardelli, Eric Witmer, Ga...",...,28360051.0,"[Christian Boxley, Jacques Nissen, Jeremy Knop...",SET_O_LINE,True,2.0,2.0,244.0,GOAL,True,False


In [28]:
df_output.to_csv("Test_Data_220806_2215.csv", index=False)

In [29]:
df_output['line_for_merge_a'] = df_output['line_players_a'].astype('str')
df_output['line_for_merge_b'] = df_output['line_players_b'].astype('str')
df_line_sets['line_players_for_merge'] = df_line_sets['line_players'].astype('str')

df_output2 = df_output.merge(
    df_line_sets.loc[:, ['line_players_for_merge', 'line_id']].rename({'line_id': 'line_a_id'}, axis=1),
    left_on='line_for_merge_a',
    right_on='line_players_for_merge',
    how='left',
    indicator=True,
    copy=True
)

df_output3 = df_output2.drop(['_merge', 'line_players_for_merge'], axis=1).merge(
    df_line_sets.loc[:, ['line_players_for_merge', 'line_id']].rename({'line_id': 'line_b_id'}, axis=1),
    left_on='line_for_merge_b',
    right_on='line_players_for_merge',
    how='left',
    indicator=True,
    copy=True
)
df_output2.drop(['line_for_merge_a', 'line_for_merge_b', 'line_players_for_merge'], axis=1, inplace=True)
df_output3.drop(['line_for_merge_a', 'line_for_merge_b', 'line_players_for_merge', '_merge'], axis=1, inplace=True)

In [30]:
df_output3.columns

df_output3.shape


(259287, 27)

In [31]:
df_output3.to_csv("Test_Data_220807_0730.csv", index=False)

In [35]:
print(df_output3.columns)

Index(['date', 'game_id', 'home_team', 'away_team', 'game_point_id',
       'current_quarter', 'time_of_score', 'team_id_a', 'line_id_a',
       'line_players_a', 'line_set_type_a', 'is_last_line_a', 'is_home_a',
       'o_point_a', 'team_id_b', 'line_id_b', 'line_players_b',
       'line_set_type_b', 'is_last_line_b', 'team_a_score_point_start',
       'team_b_score_point_start', 'scoring_team', 'score_type',
       'team_a_scored', 'team_b_scored', 'line_a_id', 'line_b_id'],
      dtype='object')


In [58]:
df_output3.loc[df_output3['line_id_b'].isna()].shape
df_output3.loc[df_output3['line_id_a'].isna()].shape

(27, 27)

In [60]:
df_line_sets.loc[df_line_sets['line_players'].isna()]

,team_id,game_id,line_players,sorted_line_players,0,1,2,3,4,5,6,line_id,line_players_for_merge


In [76]:
# Checking for weird missing lines
df_scores.columns
df_output.columns

vals_a = df_output.loc[df_output['line_id_a'].isna(), 'game_point_id'].values
vals_b = df_output.loc[df_output['line_id_b'].isna(), 'game_point_id'].values

print(vals_b)

df_scores.loc[df_scores['game_point_id'].isin(vals_b)]
# df_scores.loc[df_output['line_id_a'].isna(), 'game_point_id'].values)

[2874039 2819046 2819046]


,event_index,game_id,date,home_team,away_team,event_counter,team_id,current_quarter,time,event_type,...,point_id,our_score,their_score,line,is_home,is_away,game_point_id,home_score,away_score,scoring_team
11536,550,2874,2022-07-31,229,232,550,229,4.0,Q4 00:00,GOAL,...,39,28,9,NaN,True,False,2874039,1,0,229


In [32]:
# Making sure there are only 41 times no one scored
df_output.loc[(df_output['team_a_scored'] == False) & (df_output['team_b_scored'] == False)].shape

(390, 27)

In [33]:
df['event_type'].unique()

array([nan, 'START_OF_GAME', 'SET_D_LINE', 'PULL_INBOUNDS', 'SCORED_ON',
       'SET_O_LINE', 'POSSESSION', 'THROWAWAY', 'THROWAWAY_CAUSED',
       'GOAL', 'BLOCK', 'D_PENALTY_ON_THEM', 'D_PENALTY_ON_US',
       'OUR_MIDPOINT_TIMEOUT', 'SET_O_LINE_NO_PULL', 'END_OF_Q1',
       'THEIR_MIDPOINT_TIMEOUT', 'SET_D_LINE_NO_PULL', 'O_PENALTY_ON_US',
       'PULL_OUT_OF_BOUNDS', 'HALFTIME', 'O_PENALTY_ON_THEM', 'END_OF_Q3',
       'REF_TIMEOUT_DISCUSSION???', 'GAME_OVER', 'DROP', 'INJURY_ON_D',
       'STALL_CAUSED', 'PULL_OUR_OFFSIDES', 'INJURY_ON_O', 'END_OF_OT1',
       'STALL', 'CALLAHAN', 'CALLAHAN_THROWN'], dtype=object)